In [103]:
import random
import urlparse
import numpy as np
import joblib as jl
import xgboost as xgb

import extract
import get_rank
import get_cert
import get_whois
import get_pagerank
import get_ip
import get_sus_tld
import make_feature
from file_io import load_urls, dump_urls

In [61]:
total_mal_urls = load_urls("../std_data/mal.csv")
total_ben_urls = load_urls("../std_data/ben.csv")
total_unk_urls = load_urls("../std_data/unk.csv")
total_urls = set(total_ben_urls) | set(total_mal_urls) | set(total_unk_urls)
total_domains = list(set([urlparse.urlparse(_).hostname for _ in total_urls]))
print len(total_urls)
print len(total_domains)

2018-06-21 09:58:13 file_io.py [line:52] DEBUG	URLs Count:	14185
2018-06-21 09:58:13 file_io.py [line:52] DEBUG	URLs Count:	61595
2018-06-21 09:58:13 file_io.py [line:52] DEBUG	URLs Count:	1473


77225
9036


In [3]:
# tst_url_ip_map      = jl.load('../data/exp_rule/tst/url_ip_map.jl.z')
# tst_url_ip_dmap     = jl.load('../data/exp_rule/tst/url_ip_dmap.jl.z')
# tst_domain_rank_map = jl.load('../data/exp_rule/tst/domain_rank_map.jl.z')
# tst_domain_cert_map = jl.load('../data/exp_rule/tst/domain_cert_map.jl.z')

url_ip_map      = jl.load("../data/exp_rule/ip/url_ip_map2.jl.z" )
url_ip_dmap     = jl.load('../data/exp_rule/ip/url_ip_dmap2.jl.z')
domain_rank_map = jl.load('../data/exp_rule/alexa_rank/alexa_rank2.jl.z')
domain_cert_map = jl.load('../data/exp_rule/cert/certificate2.jl.z')

# url_ip_map.update(tst_url_ip_map)
# url_ip_dmap.update(tst_url_ip_dmap)
# domain_cert_map.update(tst_domain_cert_map)
# domain_rank_map.update(tst_domain_rank_map)

# jl.dump(url_ip_map, "../data/exp_rule/ip/url_ip_map2.jl.z")
# jl.dump(url_ip_dmap, '../data/exp_rule/ip/url_ip_dmap2.jl.z')
# jl.dump(domain_cert_map, '../data/exp_rule/cert/certificate2.jl.z')
# jl.dump(domain_rank_map, '../data/exp_rule/alexa_rank/alexa_rank2.jl.z')

domain_whois_map = get_whois.load_whois_dict(path = "../data/exp_rule/whois")
domain_pr_map = jl.load("../data/exp_rule/pr/pr.jl.z")
sus_domain = jl.load("../data/exp_rule/sus_domain/sus_domain.jl.z")

In [4]:
print len(url_ip_map)
print len(url_ip_dmap)
print len(domain_cert_map)
print len(domain_rank_map)
print ""
print len(domain_whois_map)
print len(domain_pr_map)
print len(sus_domain)

77225
12393
9036
9036

3348
9036
262


In [5]:
split_ratio = 0.99
train_ben_urls = random.sample(total_ben_urls, int(len(total_ben_urls)*split_ratio))
train_mal_urls = random.sample(total_mal_urls, int(len(total_mal_urls)*split_ratio))

test_ben_urls = list(set(total_ben_urls) - set(train_ben_urls))
test_mal_urls = list(set(total_mal_urls) - set(train_mal_urls))

print len(total_ben_urls), len(train_ben_urls), len(test_ben_urls)
print len(total_mal_urls), len(train_mal_urls), len(test_mal_urls)

61595 60979 616
14185 14043 142


In [6]:
data_obj = extract.FeatureExtract(train_mal_urls, train_ben_urls)
data_obj.feature_extract(url_ip_dmap)

In [7]:
print "ben filename", len(data_obj.ben_filename)
print "mal filename", len(data_obj.mal_filename)
print "ben filepath", len(data_obj.ben_filepath)
print "mal filepath", len(data_obj.mal_filepath)

print "mal pdomain tokens", len(data_obj.mal_pdomain_tokens)
print "mal sdomain tokens", len(data_obj.mal_sdomain_tokens)
print "ben pdomain tokens", len(data_obj.ben_pdomain_tokens)
print "ben sdomain tokens", len(data_obj.ben_sdomain_tokens)

print "ben ips", len(data_obj.mal_ips)
print "ana path", len(data_obj.domain_path_map)
print "ana port", len(data_obj.domain_port_map)

ben filename 28823
mal filename 8907
ben filepath 7560
mal filepath 754
mal pdomain tokens 148
mal sdomain tokens 139
ben pdomain tokens 823
ben sdomain tokens 745
ben ips 43
ana path 8398
ana port 123


In [8]:
feat_obj = make_feature.build_feature(data_obj, url_ip_map, url_ip_dmap, domain_whois_map, domain_rank_map, domain_cert_map, \
                                      domain_pr_map, sus_domain)

In [9]:
train_mal_x = feat_obj.build_feature_set(train_mal_urls)
train_ben_x = feat_obj.build_feature_set(train_ben_urls)
test_mal_x = feat_obj.build_feature_set(test_mal_urls)
test_ben_x = feat_obj.build_feature_set(test_ben_urls)
unk_x = feat_obj.build_feature_set(total_unk_urls)

In [10]:
train_x = np.concatenate((train_mal_x,train_ben_x), axis = 0)
train_y = [1] * len(train_mal_x) + [0] * len(train_ben_x)

test_x =  np.concatenate((test_mal_x, test_ben_x), axis = 0)
test_y = [1] * len(test_mal_x) + [0] * len(test_ben_x)

In [74]:
dtrain = xgb.DMatrix(train_x, train_y)
dtest  = xgb.DMatrix(test_x, test_y)

params={
'booster': 'gbtree',
'objective':'binary:logistic',
'gamma':0.1,  
'max_depth':12, 
'lambda':2,  
'subsample':1, 
'colsample_bytree':1, 
'min_child_weight':1, 
'silent':0 ,
'eta': 0.2, 
'seed':1000,
'nthread':8
}

watchlist  = [(dtest,'eval'), (dtrain,'train')]  
num_round = 200
bst = xgb.train(params, dtrain, num_round, watchlist) 

[0]	eval-error:0.01847	train-error:0.001719
[1]	eval-error:0.01847	train-error:0.001719
[2]	eval-error:0.01847	train-error:0.001719
[3]	eval-error:0.01847	train-error:0.001586
[4]	eval-error:0.01847	train-error:0.001453
[5]	eval-error:0.01847	train-error:0.001506
[6]	eval-error:0.01847	train-error:0.001426
[7]	eval-error:0.01847	train-error:0.001426
[8]	eval-error:0.01847	train-error:0.001386
[9]	eval-error:0.01847	train-error:0.00136
[10]	eval-error:0.01847	train-error:0.001373
[11]	eval-error:0.01847	train-error:0.001293
[12]	eval-error:0.01847	train-error:0.001253
[13]	eval-error:0.01715	train-error:0.001213
[14]	eval-error:0.01715	train-error:0.001186
[15]	eval-error:0.01715	train-error:0.001173
[16]	eval-error:0.01715	train-error:0.001146
[17]	eval-error:0.01715	train-error:0.001133
[18]	eval-error:0.01715	train-error:0.001133
[19]	eval-error:0.01715	train-error:0.001106
[20]	eval-error:0.01715	train-error:0.001106
[21]	eval-error:0.01715	train-error:0.001066
[22]	eval-error:0.017

[181]	eval-error:0.01847	train-error:0.000493
[182]	eval-error:0.01847	train-error:0.000493
[183]	eval-error:0.01847	train-error:0.000493
[184]	eval-error:0.01847	train-error:0.000493
[185]	eval-error:0.01847	train-error:0.000493
[186]	eval-error:0.01847	train-error:0.000493
[187]	eval-error:0.01847	train-error:0.000493
[188]	eval-error:0.01847	train-error:0.000493
[189]	eval-error:0.01847	train-error:0.000493
[190]	eval-error:0.01847	train-error:0.000493
[191]	eval-error:0.01847	train-error:0.000493
[192]	eval-error:0.01847	train-error:0.000493
[193]	eval-error:0.01847	train-error:0.000493
[194]	eval-error:0.01847	train-error:0.000493
[195]	eval-error:0.01847	train-error:0.000493
[196]	eval-error:0.01847	train-error:0.000493
[197]	eval-error:0.01847	train-error:0.000493
[198]	eval-error:0.01847	train-error:0.000493
[199]	eval-error:0.01847	train-error:0.000493


In [76]:
def xgb_prediction(data, urls, bst, thresh = 0.5, ntree_limit = 0):
    data = xgb.DMatrix(data, [0 for _ in range(len(data))])
    prediction = []
    y_pred = bst.predict(data, ntree_limit = ntree_limit)
    for pred, url in zip(y_pred, urls):
        if pred > thresh:
            prediction.append(url)
    print "prediction:%d\trecall:%.3f" %(len(prediction), len(prediction)/float(len(urls)))
    return prediction

ntree_limit = -1
thresh = 0.8

pred_ben_train = xgb_prediction(train_ben_x, train_ben_urls, bst, thresh = thresh, ntree_limit = ntree_limit )
pred_mal_train = xgb_prediction(train_mal_x, train_mal_urls, bst, thresh = thresh, ntree_limit = ntree_limit )
print ""
pred_ben = xgb_prediction(test_ben_x, test_ben_urls, bst, thresh = thresh, ntree_limit = ntree_limit )
pred_mal = xgb_prediction(test_mal_x, test_mal_urls, bst, thresh = thresh, ntree_limit = ntree_limit )
print ""
pred_unk = xgb_prediction(unk_x, total_unk_urls, bst, thresh = thresh, ntree_limit = ntree_limit )

prediction:0	recall:0.000
prediction:13989	recall:0.996

prediction:0	recall:0.000
prediction:127	recall:0.894

prediction:355	recall:0.241


In [79]:
bst.save_model("./model/xgb-621.model")

### extend data

In [29]:
import json
import pandas as pd

In [27]:
def load_raw_data(filepath = "../test_data/dumped_ml_result_20180620.txt"):
    with open(filepath, "r") as f:
        raw_data = [json.loads(_) for _ in f]
    raw_data = [json.loads(_["S104"]) for _ in raw_data if "S104" in _]
    raw_data = filter(lambda x:x['file_type']!=0, raw_data)
    return raw_data

raw_data = load_raw_data()

In [43]:
df = pd.DataFrame({"url":[_['url'] for _ in raw_data], "S104": [_["attr"] for _ in raw_data]})
urls = list(df.url)
print len(df)
print len(df.loc[df.S104==0])
print len(df.loc[df.S104==1])
print len(df.loc[df.S104==2])

1645
20
114
1489


In [46]:
tst_url_ip_map = get_ip.make_url_ip_map(urls)
jl.dump(tst_url_ip_map,      '../data/exp_rule/tst/url_ip_map.jl.z')

2018-06-21 09:25:30 get_ip.py [line:26] WARNING	http://oa2.gltu.cn:8889/pda/attach_show.php?P=vtvd7tjc3andprb63843v5id43&AID=39506&MODULE=email&ACCESS_PRIV=0&YM=1806&ATTACHMENT_ID=1148015073&ATTACHMENT_NAME=cbbcd5febfcecce2d7a8bcd2c6c0c9f3b2c4c1cfa3a8c6c0c9f3b1eda1a2bfcecce2bbeed2b3a3a92e7a6970&ORG_ATTACHMENT_NAME=%CB%BC%D5%FE%BF%CE%CC%E2%D7%A8%BC%D2%C6%C0%C9%F3%B2%C4%C1%CF%A3%A8%C6%C0%C9%F3%B1%ED%A1%A2%BF%CE%CC%E2%BB%EE%D2%B3%A3%A9.zip&DOWN_PRIV=1 ip request failed [Errno -3] Temporary failure in name resolution
2018-06-21 09:25:32 get_ip.py [line:26] WARNING	http://kk343.tecsc.cc/fhz2rv/armnliswqn4v/xq5sfzzjd5ts/jingdong_0031-doc ip request failed [Errno -2] Name or service not known
2018-06-21 09:27:05 get_ip.py [line:26] WARNING	http://kk343.tecsc.cc/a3vm33r1z/tpilgoq5/tcib0dpuvi/jingdong_0022-doc ip request failed [Errno -2] Name or service not known


['../data/exp_rule/tst/url_ip_map.jl.z']

In [47]:
tst_url_ip_dmap = get_ip.make_url_ip_dmap(urls)
jl.dump(tst_url_ip_dmap,     '../data/exp_rule/tst/url_ip_dmap.jl.z')

2018-06-21 09:27:47 get_ip.py [line:38] WARNING	kk343.tecsc.cc ip request failed [Errno -2] Name or service not known
2018-06-21 09:31:36 get_ip.py [line:38] WARNING	kk343.tecsc.cc ip request failed [Errno -2] Name or service not known


['../data/exp_rule/tst/url_ip_dmap.jl.z']

In [48]:
tst_domain_cert_map = get_cert.get_certificate_list(urls)
jl.dump(tst_domain_cert_map, '../data/exp_rule/tst/domain_cert_map.jl.z')

398
step:0
step:1
step:2
step:3
step:4
step:5
step:6
step:7
step:8
step:9
step:10
step:11
step:12
step:13
step:14
step:15
step:16
step:17
step:18
step:19
step:20
step:21
step:22
step:23
step:24
step:25
step:26
step:27
step:28
step:29
step:30
step:31
step:32
step:33
step:34
step:35
step:36
step:37
step:38
step:39
step:40
step:41
step:42
step:43
step:44
step:45
step:46
step:47
step:48
step:49
step:50
step:51
step:52
step:53
step:54
step:55
step:56
step:57
step:58
step:59
step:60
step:61
step:62
step:63
step:64
step:65
step:66
step:67
step:68
step:69
step:70
step:71
step:72
step:73
step:74
step:75
step:76
step:77
step:78
step:79
step:80
step:81
step:82
step:83
step:84
step:85
step:86
step:87
step:88
step:89
step:90
step:91
step:92
step:93
step:94
step:95
step:96
step:97
step:98
step:99
step:100
step:101
step:102
step:103
step:104
step:105
step:106
step:107
step:108
step:109
step:110
step:111
step:112
step:113
step:114
step:115
step:116
step:117
step:118
step:119
step:120
step:121
step:122

['../data/exp_rule/tst/domain_cert_map.jl.z']

In [49]:
tst_domain_rank_map = get_rank.get_alexa_rank_list(urls, n_jobs=2)
jl.dump(tst_domain_rank_map, '../data/exp_rule/tst/domain_rank_map.jl.z')

2018-06-21 09:46:29 get_rank.py [line:42] INFO	scheme domains count:398
2018-06-21 09:46:29 connectionpool.py [line:208] DEBUG	Starting new HTTP connection (1): xml.alexa.com
2018-06-21 09:46:29 get_rank.py [line:24] INFO	step:0
2018-06-21 09:46:29 connectionpool.py [line:208] DEBUG	Starting new HTTP connection (1): xml.alexa.com
2018-06-21 09:46:30 connectionpool.py [line:396] DEBUG	http://xml.alexa.com:80 "GET /data?cli=10&dat=nsa&ver=quirk-searchstatus&url=http://img.rrzcp8.cn HTTP/1.1" 200 301
2018-06-21 09:46:30 connectionpool.py [line:208] DEBUG	Starting new HTTP connection (1): xml.alexa.com
2018-06-21 09:46:30 connectionpool.py [line:396] DEBUG	http://xml.alexa.com:80 "GET /data?cli=10&dat=nsa&ver=quirk-searchstatus&url=http://renren.maoyun.tv HTTP/1.1" 200 464
2018-06-21 09:46:30 connectionpool.py [line:208] DEBUG	Starting new HTTP connection (1): xml.alexa.com
2018-06-21 09:46:31 connectionpool.py [line:396] DEBUG	http://xml.alexa.com:80 "GET /data?cli=10&dat=nsa&ver=quirk-se

2018-06-21 09:46:42 connectionpool.py [line:208] DEBUG	Starting new HTTP connection (1): xml.alexa.com
2018-06-21 09:46:43 connectionpool.py [line:396] DEBUG	http://xml.alexa.com:80 "GET /data?cli=10&dat=nsa&ver=quirk-searchstatus&url=http://niagho-img.ccnapps.cn HTTP/1.1" 200 325
2018-06-21 09:46:43 connectionpool.py [line:208] DEBUG	Starting new HTTP connection (1): xml.alexa.com
2018-06-21 09:46:43 connectionpool.py [line:396] DEBUG	http://xml.alexa.com:80 "GET /data?cli=10&dat=nsa&ver=quirk-searchstatus&url=http://llcdnr2.linkinme.com HTTP/1.1" 200 322
2018-06-21 09:46:43 connectionpool.py [line:208] DEBUG	Starting new HTTP connection (1): xml.alexa.com
2018-06-21 09:46:44 connectionpool.py [line:396] DEBUG	http://xml.alexa.com:80 "GET /data?cli=10&dat=nsa&ver=quirk-searchstatus&url=http://avn01.speednavi.co.kr HTTP/1.1" 200 740
2018-06-21 09:46:44 connectionpool.py [line:208] DEBUG	Starting new HTTP connection (1): xml.alexa.com
2018-06-21 09:46:44 connectionpool.py [line:396] DEB

2018-06-21 09:47:07 connectionpool.py [line:396] DEBUG	http://xml.alexa.com:80 "GET /data?cli=10&dat=nsa&ver=quirk-searchstatus&url=http://patch2.51lg.com HTTP/1.1" 200 307
2018-06-21 09:47:07 connectionpool.py [line:208] DEBUG	Starting new HTTP connection (1): xml.alexa.com
2018-06-21 09:47:08 connectionpool.py [line:396] DEBUG	http://xml.alexa.com:80 "GET /data?cli=10&dat=nsa&ver=quirk-searchstatus&url=http://21576.xc.iziyo.com HTTP/1.1" 200 452
2018-06-21 09:47:08 connectionpool.py [line:208] DEBUG	Starting new HTTP connection (1): xml.alexa.com
2018-06-21 09:47:09 connectionpool.py [line:396] DEBUG	http://xml.alexa.com:80 "GET /data?cli=10&dat=nsa&ver=quirk-searchstatus&url=http://cdn.wiair.com HTTP/1.1" 200 414
2018-06-21 09:47:09 connectionpool.py [line:208] DEBUG	Starting new HTTP connection (1): xml.alexa.com
2018-06-21 09:47:10 connectionpool.py [line:396] DEBUG	http://xml.alexa.com:80 "GET /data?cli=10&dat=nsa&ver=quirk-searchstatus&url=http://h8ytj9vt.dodobao.net HTTP/1.1" 2

2018-06-21 09:47:30 connectionpool.py [line:208] DEBUG	Starting new HTTP connection (1): xml.alexa.com
2018-06-21 09:47:30 connectionpool.py [line:396] DEBUG	http://xml.alexa.com:80 "GET /data?cli=10&dat=nsa&ver=quirk-searchstatus&url=http://41736.5463d.aqwxrfghh.com HTTP/1.1" 200 337
2018-06-21 09:47:30 connectionpool.py [line:208] DEBUG	Starting new HTTP connection (1): xml.alexa.com
2018-06-21 09:47:31 connectionpool.py [line:396] DEBUG	http://xml.alexa.com:80 "GET /data?cli=10&dat=nsa&ver=quirk-searchstatus&url=http://pc6.down.123ch.cn HTTP/1.1" 200 470
2018-06-21 09:47:31 connectionpool.py [line:208] DEBUG	Starting new HTTP connection (1): xml.alexa.com
2018-06-21 09:47:32 connectionpool.py [line:396] DEBUG	http://xml.alexa.com:80 "GET /data?cli=10&dat=nsa&ver=quirk-searchstatus&url=http://join.huohuotuan.cn HTTP/1.1" 200 316
2018-06-21 09:47:32 connectionpool.py [line:208] DEBUG	Starting new HTTP connection (1): xml.alexa.com
2018-06-21 09:47:33 connectionpool.py [line:396] DEBUG

2018-06-21 09:47:53 connectionpool.py [line:396] DEBUG	http://xml.alexa.com:80 "GET /data?cli=10&dat=nsa&ver=quirk-searchstatus&url=http://map1.mini1.cn HTTP/1.1" 200 461
2018-06-21 09:47:53 connectionpool.py [line:208] DEBUG	Starting new HTTP connection (1): xml.alexa.com
2018-06-21 09:48:00 connectionpool.py [line:396] DEBUG	http://xml.alexa.com:80 "GET /data?cli=10&dat=nsa&ver=quirk-searchstatus&url=http://player.yswhw.cn HTTP/1.1" 200 419
2018-06-21 09:48:00 connectionpool.py [line:208] DEBUG	Starting new HTTP connection (1): xml.alexa.com
2018-06-21 09:48:00 connectionpool.py [line:396] DEBUG	http://xml.alexa.com:80 "GET /data?cli=10&dat=nsa&ver=quirk-searchstatus&url=http://94831.5463d.aqwxrfghh.com HTTP/1.1" 200 337
2018-06-21 09:48:00 connectionpool.py [line:208] DEBUG	Starting new HTTP connection (1): xml.alexa.com
2018-06-21 09:48:01 connectionpool.py [line:396] DEBUG	http://xml.alexa.com:80 "GET /data?cli=10&dat=nsa&ver=quirk-searchstatus&url=http://map4.mini1.cn HTTP/1.1" 2

...

2018-06-21 09:49:59 connectionpool.py [line:396] DEBUG	http://xml.alexa.com:80 "GET /data?cli=10&dat=nsa&ver=quirk-searchstatus&url=http://www.qqcdo.com HTTP/1.1" 200 379
2018-06-21 09:49:59 connectionpool.py [line:208] DEBUG	Starting new HTTP connection (1): xml.alexa.com
2018-06-21 09:49:59 connectionpool.py [line:396] DEBUG	http://xml.alexa.com:80 "GET /data?cli=10&dat=nsa&ver=quirk-searchstatus&url=http://static1.keepcdn.com HTTP/1.1" 200 437
2018-06-21 09:49:59 connectionpool.py [line:208] DEBUG	Starting new HTTP connection (1): xml.alexa.com
2018-06-21 09:50:00 connectionpool.py [line:396] DEBUG	http://xml.alexa.com:80 "GET /data?cli=10&dat=nsa&ver=quirk-searchstatus&url=http://r6qoc.52gyxz.ynxadh.cn HTTP/1.1" 200 420
2018-06-21 09:50:00 connectionpool.py [line:208] DEBUG	Starting new HTTP connection (1): xml.alexa.com
2018-06-21 09:50:00 connectionpool.py [line:396] DEBUG	http://xml.alexa.com:80 "GET /data?cli=10&dat=nsa&ver=quirk-searchstatus&url=http://www.763va.com HTTP/1.1" 

2018-06-21 09:50:10 connectionpool.py [line:208] DEBUG	Starting new HTTP connection (1): xml.alexa.com
2018-06-21 09:50:11 connectionpool.py [line:396] DEBUG	http://xml.alexa.com:80 "GET /data?cli=10&dat=nsa&ver=quirk-searchstatus&url=http://ddz.dl.tuyoo.com HTTP/1.1" 200 426
2018-06-21 09:50:11 connectionpool.py [line:208] DEBUG	Starting new HTTP connection (1): xml.alexa.com
2018-06-21 09:50:11 connectionpool.py [line:396] DEBUG	http://xml.alexa.com:80 "GET /data?cli=10&dat=nsa&ver=quirk-searchstatus&url=http://gogo.applekid.cn HTTP/1.1" 200 310
2018-06-21 09:50:11 connectionpool.py [line:208] DEBUG	Starting new HTTP connection (1): xml.alexa.com
2018-06-21 09:50:12 connectionpool.py [line:396] DEBUG	http://xml.alexa.com:80 "GET /data?cli=10&dat=nsa&ver=quirk-searchstatus&url=http://updatecdn.k8.lytdgs.cn HTTP/1.1" 200 328
2018-06-21 09:50:12 connectionpool.py [line:208] DEBUG	Starting new HTTP connection (1): xml.alexa.com
2018-06-21 09:50:13 connectionpool.py [line:396] DEBUG	http:

2018-06-21 09:50:22 connectionpool.py [line:396] DEBUG	http://xml.alexa.com:80 "GET /data?cli=10&dat=nsa&ver=quirk-searchstatus&url=http://sci-hub.tw HTTP/1.1" 200 443
2018-06-21 09:50:22 connectionpool.py [line:208] DEBUG	Starting new HTTP connection (1): xml.alexa.com
2018-06-21 09:50:22 connectionpool.py [line:396] DEBUG	http://xml.alexa.com:80 "GET /data?cli=10&dat=nsa&ver=quirk-searchstatus&url=http://qiniu.chengdaolc.com HTTP/1.1" 200 322
2018-06-21 09:50:22 connectionpool.py [line:208] DEBUG	Starting new HTTP connection (1): xml.alexa.com
2018-06-21 09:50:24 connectionpool.py [line:396] DEBUG	http://xml.alexa.com:80 "GET /data?cli=10&dat=nsa&ver=quirk-searchstatus&url=http://35720.5463d.aqwxrfghh.com HTTP/1.1" 200 337
2018-06-21 09:50:24 connectionpool.py [line:208] DEBUG	Starting new HTTP connection (1): xml.alexa.com
2018-06-21 09:50:25 connectionpool.py [line:396] DEBUG	http://xml.alexa.com:80 "GET /data?cli=10&dat=nsa&ver=quirk-searchstatus&url=http://item.ifreetalk.com HTTP

2018-06-21 09:50:39 connectionpool.py [line:208] DEBUG	Starting new HTTP connection (1): xml.alexa.com
2018-06-21 09:50:40 connectionpool.py [line:396] DEBUG	http://xml.alexa.com:80 "GET /data?cli=10&dat=nsa&ver=quirk-searchstatus&url=http://125013.5463d.aqwxrfghh.com HTTP/1.1" 200 340
2018-06-21 09:50:40 connectionpool.py [line:208] DEBUG	Starting new HTTP connection (1): xml.alexa.com
2018-06-21 09:50:42 connectionpool.py [line:396] DEBUG	http://xml.alexa.com:80 "GET /data?cli=10&dat=nsa&ver=quirk-searchstatus&url=http://epub.ds.ireader.com HTTP/1.1" 200 634
2018-06-21 09:50:42 connectionpool.py [line:208] DEBUG	Starting new HTTP connection (1): xml.alexa.com
2018-06-21 09:50:42 connectionpool.py [line:396] DEBUG	http://xml.alexa.com:80 "GET /data?cli=10&dat=nsa&ver=quirk-searchstatus&url=http://iuqmjzpt.ahhxdl.cn HTTP/1.1" 200 316
2018-06-21 09:50:42 connectionpool.py [line:208] DEBUG	Starting new HTTP connection (1): xml.alexa.com
2018-06-21 09:50:43 connectionpool.py [line:396] DE

['../data/exp_rule/tst/domain_rank_map.jl.z']

In [51]:
domain_whois_map_update = get_whois.get_whois_list(urls, "../data/exp_rule/whois/", check=True, n_jobs=4)

2018-06-21 09:52:11 get_whois.py [line:64] INFO	primary_domain count:223
2018-06-21 09:52:11 get_whois.py [line:31] INFO	step:0
2018-06-21 09:52:39 get_whois.py [line:42] ERROR	get whois info error 123.249.9.113. [Errno -2] Name or service not known
2018-06-21 09:52:51 get_whois.py [line:42] ERROR	get whois info error guiwa.vip [Errno -2] Name or service not known


In [63]:
sus_domain_update = get_sus_tld.extract_sus_url(urls)
sus_domain = sus_domain | sus_domain_update
jl.dump(sus_domain, "../data/exp_rule/sus_domain/sus_domain.jl.z")

domain cnt:398


['../data/exp_rule/sus_domain/sus_domain.jl.z']

In [62]:
urls = [url.encode("utf-8") for url in urls]
total_urls_add = total_urls | set(urls)
print len(total_urls)
print len(total_urls_add)

77225
78756


In [64]:
domain_pr_map = get_pagerank.get_pr_list(total_urls_add, pr_dict=domain_pr_map)
jl.dump(domain_pr_map, "../data/exp_rule/pr/pr.jl.z")

2018-06-21 09:59:11 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): pr.aizhan.com
2018-06-21 09:59:11 connectionpool.py [line:396] DEBUG	https://pr.aizhan.com:443 "GET /i7km.ebyymzbp.cn/ HTTP/1.1" 200 None
2018-06-21 09:59:13 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): pr.aizhan.com
2018-06-21 09:59:13 connectionpool.py [line:396] DEBUG	https://pr.aizhan.com:443 "GET /down39.bizport.cn/ HTTP/1.1" 200 None
2018-06-21 09:59:14 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): pr.aizhan.com
2018-06-21 09:59:14 connectionpool.py [line:396] DEBUG	https://pr.aizhan.com:443 "GET /29y00.ebyymzbp.cn/ HTTP/1.1" 200 None
2018-06-21 09:59:15 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): pr.aizhan.com
2018-06-21 09:59:21 connectionpool.py [line:396] DEBUG	https://pr.aizhan.com:443 "GET /19iwz.ebyymzbp.cn/ HTTP/1.1" 200 None
2018-06-21 09:59:22 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection 

2018-06-21 10:00:32 connectionpool.py [line:396] DEBUG	https://pr.aizhan.com:443 "GET /idhhje.shengfanglvshi.com/ HTTP/1.1" 200 None
2018-06-21 10:00:33 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): pr.aizhan.com
2018-06-21 10:00:34 connectionpool.py [line:396] DEBUG	https://pr.aizhan.com:443 "GET /lei80220uao.asoyhmje.cn/ HTTP/1.1" 200 None
2018-06-21 10:00:35 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): pr.aizhan.com
2018-06-21 10:00:36 connectionpool.py [line:396] DEBUG	https://pr.aizhan.com:443 "GET /ttd.sharpensoft.com/ HTTP/1.1" 200 None
2018-06-21 10:00:37 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): pr.aizhan.com
2018-06-21 10:00:37 connectionpool.py [line:396] DEBUG	https://pr.aizhan.com:443 "GET /gogo.applekid.cn/ HTTP/1.1" 200 None
2018-06-21 10:00:38 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): pr.aizhan.com
2018-06-21 10:00:39 connectionpool.py [line:396] DEBUG	https://pr.aiz

2018-06-21 10:01:49 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): pr.aizhan.com
2018-06-21 10:01:50 connectionpool.py [line:396] DEBUG	https://pr.aizhan.com:443 "GET /down55.bizport.cn/ HTTP/1.1" 200 None
2018-06-21 10:01:51 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): pr.aizhan.com
2018-06-21 10:01:52 connectionpool.py [line:396] DEBUG	https://pr.aizhan.com:443 "GET /21589.xc.mieseng.com/ HTTP/1.1" 200 None
2018-06-21 10:01:53 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): pr.aizhan.com
2018-06-21 10:01:53 connectionpool.py [line:396] DEBUG	https://pr.aizhan.com:443 "GET /down.263209.com/ HTTP/1.1" 200 None
2018-06-21 10:01:54 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): pr.aizhan.com
2018-06-21 10:01:54 connectionpool.py [line:396] DEBUG	https://pr.aizhan.com:443 "GET /21597.xc.tduou.com/ HTTP/1.1" 200 None
2018-06-21 10:01:55 connectionpool.py [line:824] DEBUG	Starting new HTTPS connecti

2018-06-21 10:02:56 connectionpool.py [line:396] DEBUG	https://pr.aizhan.com:443 "GET /kk343.tecsc.cc/ HTTP/1.1" 200 None
2018-06-21 10:02:57 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): pr.aizhan.com
2018-06-21 10:02:58 connectionpool.py [line:396] DEBUG	https://pr.aizhan.com:443 "GET /www.469t.com/ HTTP/1.1" 200 None
2018-06-21 10:02:59 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): pr.aizhan.com
2018-06-21 10:03:00 connectionpool.py [line:396] DEBUG	https://pr.aizhan.com:443 "GET /24441.5463d.aqwxrfghh.com/ HTTP/1.1" 200 None
2018-06-21 10:03:01 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): pr.aizhan.com
2018-06-21 10:03:02 connectionpool.py [line:396] DEBUG	https://pr.aizhan.com:443 "GET /21587.xc.tduou.com/ HTTP/1.1" 200 None
2018-06-21 10:03:03 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): pr.aizhan.com
2018-06-21 10:03:03 connectionpool.py [line:396] DEBUG	https://pr.aizhan.com:443 "G

2018-06-21 10:03:58 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): pr.aizhan.com
2018-06-21 10:03:59 connectionpool.py [line:396] DEBUG	https://pr.aizhan.com:443 "GET /hkvd3.tp33.net/ HTTP/1.1" 200 None
2018-06-21 10:04:00 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): pr.aizhan.com
2018-06-21 10:04:00 connectionpool.py [line:396] DEBUG	https://pr.aizhan.com:443 "GET /05259.5463d.aqwxrfghh.com/ HTTP/1.1" 200 None
2018-06-21 10:04:01 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): pr.aizhan.com
2018-06-21 10:04:02 connectionpool.py [line:396] DEBUG	https://pr.aizhan.com:443 "GET /www.fncgtc-gz.com/ HTTP/1.1" 200 None
2018-06-21 10:04:03 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): pr.aizhan.com
2018-06-21 10:04:08 connectionpool.py [line:396] DEBUG	https://pr.aizhan.com:443 "GET /21592.xc.iziyo.com/ HTTP/1.1" 200 None
2018-06-21 10:04:09 connectionpool.py [line:824] DEBUG	Starting new HTTPS conn

2018-06-21 10:05:12 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): pr.aizhan.com
2018-06-21 10:05:13 connectionpool.py [line:396] DEBUG	https://pr.aizhan.com:443 "GET /xiazai.photozoomchina.com/ HTTP/1.1" 200 None
2018-06-21 10:05:14 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): pr.aizhan.com
2018-06-21 10:05:14 connectionpool.py [line:396] DEBUG	https://pr.aizhan.com:443 "GET /i7k2.ebyymzbp.cn/ HTTP/1.1" 200 None
2018-06-21 10:05:15 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): pr.aizhan.com
2018-06-21 10:05:16 connectionpool.py [line:396] DEBUG	https://pr.aizhan.com:443 "GET /mcresource.huyagame.cn/ HTTP/1.1" 200 None
2018-06-21 10:05:17 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): pr.aizhan.com
2018-06-21 10:05:17 connectionpool.py [line:396] DEBUG	https://pr.aizhan.com:443 "GET /21597.xc.iziyo.com/ HTTP/1.1" 200 None
2018-06-21 10:05:18 connectionpool.py [line:824] DEBUG	Starting new HTT

2018-06-21 10:06:34 connectionpool.py [line:396] DEBUG	https://pr.aizhan.com:443 "GET /www.763va.com/ HTTP/1.1" 200 None
2018-06-21 10:06:35 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): pr.aizhan.com
2018-06-21 10:06:36 connectionpool.py [line:396] DEBUG	https://pr.aizhan.com:443 "GET /down52.bizport.cn/ HTTP/1.1" 200 None
2018-06-21 10:06:37 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): pr.aizhan.com
2018-06-21 10:06:37 connectionpool.py [line:396] DEBUG	https://pr.aizhan.com:443 "GET /41736.5463d.aqwxrfghh.com/ HTTP/1.1" 200 None
2018-06-21 10:06:38 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): pr.aizhan.com
2018-06-21 10:06:39 connectionpool.py [line:396] DEBUG	https://pr.aizhan.com:443 "GET /21597.xc.cangpie.com/ HTTP/1.1" 200 None
2018-06-21 10:06:40 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): pr.aizhan.com
2018-06-21 10:06:40 connectionpool.py [line:396] DEBUG	https://pr.aizhan.com:

['../data/exp_rule/pr/pr.jl.z']

### reload data and make prediction

In [65]:
tst_url_ip_map      = jl.load('../data/exp_rule/tst/url_ip_map.jl.z')
tst_url_ip_dmap     = jl.load('../data/exp_rule/tst/url_ip_dmap.jl.z')
tst_domain_rank_map = jl.load('../data/exp_rule/tst/domain_rank_map.jl.z')
tst_domain_cert_map = jl.load('../data/exp_rule/tst/domain_cert_map.jl.z')

url_ip_map      = jl.load("../data/exp_rule/ip/url_ip_map2.jl.z" )
url_ip_dmap     = jl.load('../data/exp_rule/ip/url_ip_dmap2.jl.z')
domain_rank_map = jl.load('../data/exp_rule/alexa_rank/alexa_rank2.jl.z')
domain_cert_map = jl.load('../data/exp_rule/cert/certificate2.jl.z')

url_ip_map.update(tst_url_ip_map)
url_ip_dmap.update(tst_url_ip_dmap)
domain_cert_map.update(tst_domain_cert_map)
domain_rank_map.update(tst_domain_rank_map)

jl.dump(url_ip_map, "../data/exp_rule/ip/url_ip_map2.jl.z")
jl.dump(url_ip_dmap, '../data/exp_rule/ip/url_ip_dmap2.jl.z')
jl.dump(domain_cert_map, '../data/exp_rule/cert/certificate2.jl.z')
jl.dump(domain_rank_map, '../data/exp_rule/alexa_rank/alexa_rank2.jl.z')

domain_whois_map = get_whois.load_whois_dict(path = "../data/exp_rule/whois")
domain_pr_map = jl.load("../data/exp_rule/pr/pr.jl.z")
sus_domain = jl.load("../data/exp_rule/sus_domain/sus_domain.jl.z")

In [66]:
print len(url_ip_map)
print len(url_ip_dmap)
print len(domain_cert_map)
print len(domain_rank_map)
print ""
print len(domain_whois_map)
print len(domain_pr_map)
print len(sus_domain)

78756
12624
9268
9268

3411
9268
267


In [80]:
bst2 = xgb.Booster(model_file="./model/xgb-621.model")

In [84]:
feat_obj2 = make_feature.build_feature(data_obj, url_ip_map, url_ip_dmap, domain_whois_map, domain_rank_map, domain_cert_map, \
                                      domain_pr_map, sus_domain)
tst_x = feat_obj2.build_feature_set(urls)

dtest  = xgb.DMatrix(tst_x, [0]*len(tst_x))

In [86]:
predict_prob = bst2.predict(dtest)

In [87]:
thresh = 0.8
predict = [url for url, score in zip(urls, predict_prob) if score > thresh]

In [89]:
len(predict)

149

In [104]:
dump_urls(predict, "0621-xgb.csv")

2018-06-21 11:07:26 file_io.py [line:33] DEBUG	URLs has been dump	0621-xgb.csv


### validate

In [162]:
pred = predict

dft_url, dft_res, nfp_urls, nfp_sha256 = jl.load("../data/exp_rule/df_res6.jl.z")
test_sample = list(set(pred) - set(dft_url.url))
print len(test_sample)

0


In [163]:
df_info = pd.concat([dft_url.loc[dft_url.url == _] for _ in pred])
df_tres = pd.concat([dft_res.loc[dft_res.sha256 == _] for _ in set(df_info.sha256)])
df_tres = df_tres.drop_duplicates('sha256')
df_fp = df_tres.loc[df_tres.positives == 0]
fp_urls = []

try:
    fp_urls = list(pd.concat([df_info.loc[df_info.sha256 == _] for _ in df_fp.sha256]).url)
except:
    fp_urls = []

fp_urls = list(set(fp_urls) - set(nfp_urls))
fp_sha256 = set(df_fp.sha256) - set(nfp_sha256)

print "unique sha256:\t%d, %d" %(len(set(df_tres.sha256)), len(set(df_tres.sha256))-len(set(df_fp.sha256)))
print "fp sha256:%d\tfp url:%d" %(len(fp_sha256), len(fp_urls))
print ""
print "positives ==-1:\t%d" %len(set(df_tres.loc[df_tres.positives == -1].sha256))
print "positives == 0:\t%d" %len(fp_sha256)
print "positives < 5:\t%d"  %(len(set(df_tres.loc[df_tres.positives <  5].sha256)) - len(set(df_tres.loc[df_tres.positives <= 0].sha256)))
print "positives 5~10:\t%d"  %(len(set(df_tres.loc[df_tres.positives <  10].sha256)) - len(set(df_tres.loc[df_tres.positives <  5].sha256)))
print "positives >=10:\t%d" %len(set(df_tres.loc[df_tres.positives >= 10].sha256))

unique sha256:	21, 20
fp sha256:1	fp url:1

positives ==-1:	0
positives == 0:	1
positives < 5:	0
positives 5~10:	2
positives >=10:	18


In [164]:
positives = [int(df_tres.loc[df_tres.sha256 == _].positives.values) for _ in list(df_info.sha256)]
df_info["positives"] = positives

In [170]:
df_info.to_csv("./0621-xgb-url-detection.csv")
df_tres.to_csv("./0621-xgb-file-detection.csv")

In [165]:
df_temp = df_info.drop_duplicates("sha256")
test_sample = list(df_temp.loc[df_temp.positives==-1].url)

In [166]:
import validate
reload(validate)
valid_obj = validate.ValidateWithVT(output_path = "/home/sparrow/sparrow/data/edr_test/testx")
df_url, df_res = valid_obj.do_validate_file(test_sample, upload=False, sleep=0)

2018-06-21 14:25:39 validate.py [line:191] INFO	remove file from /home/sparrow/sparrow/data/edr_test/testx


In [159]:
def update_validate_sha256(dft_res, df_res):
    dft_res = dft_res.drop_duplicates("sha256")
    sha256 = list(set(dft_res.sha256))
    sha256 = [_ for _ in sha256 if _ != None and _ not in set(df_res.sha256)]
    dft_res = pd.concat([dft_res.loc[dft_res.sha256 == _] for _ in sha256])
    return dft_res

def update_validate_url(dft_url, df_url):
    dft_url = dft_url.drop_duplicates("url")
    urls = list(set(dft_url.url))
    urls = [_ for _ in urls if _ not in set(df_url.url)]
    dft_url = pd.concat([dft_url.loc[dft_url.url == _] for _ in urls])
    return dft_url

In [160]:
dft_res = update_validate_sha256(dft_res, df_res)
dft_url = update_validate_url(dft_url, df_url)

In [161]:
dfs_url = pd.concat([dft_url, df_url])
dfs_res = pd.concat([dft_res, df_res])
jl.dump([dfs_url, dfs_res, nfp_urls, nfp_sha256], "../data/exp_rule/df_res6.jl.z")

['../data/exp_rule/df_res6.jl.z']